In [180]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import os
import datetime
from sklearn.model_selection import train_test_split

In [181]:
data = pd.read_pickle('DR_Data/rentals.pickle')

X=np.array([data.pickup_zone, data.month, data.weekday, data.air_temperature, data.rain_duration, data.rain_intensity, data.GHI]).T
y=np.array(data.dropoff_zone).T
'''
y = data.dropoff_zone
X = data.drop(['created_at','finished_at','pickup_hub_id','dropoff_hub_id','user_id','dropoff_zone'], axis=1)
'''
X

array([[35.        ,  3.        ,  6.        , ..., -0.34839599,
        -0.3022235 , -0.67088727],
       [ 0.        ,  3.        ,  6.        , ..., -0.34839599,
        -0.3022235 , -0.67088727],
       [77.        ,  3.        ,  6.        , ..., -0.34839599,
        -0.3022235 , -0.67088727],
       ...,
       [44.        ,  4.        ,  1.        , ..., -0.35568365,
        -0.3022235 ,  0.79119077],
       [77.        ,  4.        ,  1.        , ..., -0.35568365,
        -0.3022235 ,  0.79119077],
       [33.        ,  4.        ,  1.        , ..., -0.35568365,
        -0.3022235 ,  0.79119077]])

In [182]:
# one hot encoding
num_zones = int(max(X[:,0])+1)
num_months = int(max(X[:,1])+1)
num_days = int(max(X[:,2])+1)
weather=True

# RBF Encoding
RBF_Encoding=False
if RBF_Encoding:
    Euc_distances_sq=np.load('DR_Data/Zone_distances.npy')
    #Gamma = 1/2*sigma^2 and is a tuning parameter
    gamma=0.1
    Euc_distances=np.exp(-Euc_distances_sq*gamma)

num_features = int(num_zones+num_months+num_days)
if weather:
    X_enc=np.zeros((int(X.shape[0]),num_features+4))
else:
    X_enc=np.zeros((int(X.shape[0]),num_features))
for i in range(len(X)):
    i=int(i)
    if RBF_Encoding:
        X_enc[i][:num_zones] = Euc_distances_sq[int(X[i,0])] # zone
    else:
        X_enc[i][int(X[i,0])] = 1 # zone
    X_enc[i][int(X[i,1])+num_zones] = 1 # month
    X_enc[i][int(X[i,2])+num_zones+num_months] = 1 # days
    if weather:
        X_enc[i,num_features:]=X[i,3:] #Encoding weather data

In [183]:
X_train, X_valid, y_train, y_valid = train_test_split(X_enc, y, test_size=0.2, random_state=1)

In [184]:
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

In [185]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

No GPU available.


**BUILDING MODEL**

In [186]:
# Hyperparameters
num_classes = num_zones
num_l1 = 4096
num_l2 = 2048
num_l3 = 512
num_l4 = 128

learning_rate = 1e-4
weight_decay = 0

class Net(nn.Module):
    def __init__(self, num_features, num_hidden1, num_hidden2, num_hidden3, num_hidden4, num_output):
        super(Net, self).__init__()
        # input layer
        if weather:
            self.W_1 = Parameter(init.kaiming_normal_(torch.Tensor(num_hidden1, num_features+4)))
        else:
            self.W_1 = Parameter(init.kaiming_normal_(torch.Tensor(num_hidden1, num_features)))
        self.b_1 = Parameter(init.constant_(torch.Tensor(num_hidden1), 0))
        # hidden layer 1
        self.W_2 = Parameter(init.kaiming_normal_(torch.Tensor(num_hidden2, num_hidden1)))
        self.b_2 = Parameter(init.constant_(torch.Tensor(num_hidden2), 0))
        # hidden layer 2
        self.W_3 = Parameter(init.kaiming_normal_(torch.Tensor(num_hidden3, num_hidden2)))
        self.b_3 = Parameter(init.constant_(torch.Tensor(num_hidden3), 0))
        # hidden layer 3
        self.W_4 = Parameter(init.kaiming_normal_(torch.Tensor(num_hidden4, num_hidden3)))
        self.b_4 = Parameter(init.constant_(torch.Tensor(num_hidden4), 0))
         # hidden layer 4
        self.W_5 = Parameter(init.kaiming_normal_(torch.Tensor(num_output, num_hidden4)))
        self.b_5 = Parameter(init.constant_(torch.Tensor(num_output), 0))
        # define activation function in constructor
        self.activation = torch.nn.Tanh()
        # define dropout to reduce overfitting
        self.drop = torch.nn.Dropout(p=0.5)
        # define batchnorm
        self.batch1 = torch.nn.BatchNorm1d(num_hidden1)
        
    def forward(self, x):
        # forward prop
        x=x.type(torch.FloatTensor)
        x = F.linear(x, self.W_1, self.b_1)
        x = self.activation(x)
        x = self.drop(x)
        x = self.batch1(x)
        x = F.linear(x, self.W_2, self.b_2)
        x = self.activation(x)
        x = F.linear(x, self.W_3, self.b_3)
        x = self.activation(x)
        x = F.linear(x, self.W_4, self.b_4)
        x = self.activation(x)
        x = F.linear(x, self.W_5, self.b_5)
        # dropout
        
        return F.softmax(x, dim=1)


net = Net(num_features, num_l1, num_l2,num_l3, num_l4, num_classes)
if use_cuda:
    net.cuda()
    
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import accuracy_score

# setting hyperparameters and gettings epoch sizes
batch_size = 10000
num_epochs = 100
num_samples_train = X_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = X_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size

# setting up lists for handling loss/accuracy
train_acc, train_loss = [], []
valid_acc, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
losses = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss = 0
    net.train()
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        x_batch = get_variable(Variable(torch.from_numpy(X_train[slce])))
        output = net(x_batch)
        
        # compute gradients given loss
        target_batch = get_variable(Variable(torch.from_numpy(y_train[slce]).long()))
        batch_loss = criterion(output, target_batch)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        cur_loss += batch_loss   
    losses.append(cur_loss / batch_size)

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        x_batch = get_variable(Variable(torch.from_numpy(X_train[slce])))
        
        output = net(x_batch)
        preds = torch.max(output, 1)[1]
        
        train_targs += list(y_train[slce])
        train_preds += list(preds.data.numpy())
    
    ### Evaluate validation
    val_preds, val_targs = [], []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        x_batch = get_variable(Variable(torch.from_numpy(X_valid[slce])))
        
        output = net(x_batch)
        preds = torch.max(output, 1)[1]
        val_preds += list(preds.data.numpy())
        val_targs += list(y_valid[slce])

    train_acc_cur = accuracy_score(train_targs, train_preds)
    valid_acc_cur = accuracy_score(val_targs, val_preds)
    
    train_acc.append(train_acc_cur)
    valid_acc.append(valid_acc_cur)
    
    if epoch % 1 == 0:
        print("Epoch %2i : Train Loss %f , Train acc %f, Valid acc %f" % (
                epoch+1, losses[-1], train_acc_cur, valid_acc_cur))

epoch = np.arange(len(train_acc))
plt.figure()
plt.plot(epoch, train_acc, 'r', epoch, valid_acc, 'b')
plt.legend(['Train Accucary','Validation Accuracy'])
plt.xlabel('Updates'), plt.ylabel('Acc')

Epoch  1 : Train Loss 0.008223 , Train acc 0.163772, Valid acc 0.160200
Epoch  2 : Train Loss 0.008023 , Train acc 0.277022, Valid acc 0.277075
Epoch  3 : Train Loss 0.007894 , Train acc 0.307478, Valid acc 0.307100
Epoch  4 : Train Loss 0.007820 , Train acc 0.315944, Valid acc 0.315650


In [ ]:
#torch.save(net,'FFNN_months_days.pt')
#output = net(get_variable(Variable(torch.from_numpy(X_valid_enc))))
#plt.plot(output[10].detach().numpy())
#print(np.argmax(X_valid_enc[0]))
#print(y_train[10])